# Data Science Section - Phase 3
# Author: Peter Lee

1. Loading dataset

In [132]:
import pickle
import numpy as np
import pandas as pd

def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

batch1 = unpickle('./data/data_batch_1')
batch2 = unpickle('./data/data_batch_2')
batch3 = unpickle('./data/data_batch_3')
batch4 = unpickle('./data/data_batch_4')
batch5 = unpickle('./data/data_batch_5')
test = unpickle('./data/test_batch')

2. Exploratory Data Analaysis

In [133]:
# Each batch contains 10000 images with each image storing a 32x32 colour image. The first 1024 contains the red channel values, the next 1024 contains the green, and the final contains the blue.
# Therefore each batch should have a shape of 10000x3072.

train1_x = batch1[b'data']
print(train1_x.shape)
train1_y = batch1[b'labels']

train2_x = batch2[b'data']
print(train2_x.shape)
train2_y = batch2[b'labels']

train3_x = batch3[b'data']
print(train3_x.shape)
train3_y = batch3[b'labels']

train4_x = batch4[b'data']
print(train4_x.shape)
train4_y = batch4[b'labels']

train5_x = batch5[b'data']
print(train5_x.shape)
train5_y = batch5[b'labels']

# Combining all the batches into one list. Since each array will be concatenated in order, the indeces for labels with the corresponding images will stay the same.

train_x = np.concatenate((train1_x, train2_x, train3_x, train4_x, train5_x))
train_y = np.concatenate((train1_y, train2_y, train3_y, train4_y, train5_y))

test_x = test[b'data']
print(test_x.shape)
test_y = test[b'labels']

# Now we have all the required data and labels to be able to train.

(10000, 3072)
(10000, 3072)
(10000, 3072)
(10000, 3072)
(10000, 3072)
(10000, 3072)


In [ ]:
print(train_x.shape)
print(train_y.shape)
print(test_y)



In [129]:
from PIL import Image

#Producing the first 5 images from the training batch.

for numImg in range(5):
    w, h = 32, 32
    data = np.zeros((h, w, 3), dtype=np.uint8)
    for i in range(32):
        for j in range(32):
            data[i][j] = [train_x[numImg][i * 32 + j], train_x[numImg][i * 32 + j + 1024], train_x[numImg][i * 32 + j + 2048]]

    img = Image.fromarray(data, 'RGB')
    img.save(f"img{numImg}.png")
    img.show()

The label I will be choosing is "ship" which is a value of 8.

# 3. Data Processing

In [146]:
filter_train_x = []
filter_train_y = []
filter_test_x = []
filter_test_y = []

for i in range(len(train_y)):
    if(train_y[i] == 8):
        filter_train_x.append(train_x[i])
        filter_train_y.append(train_y[i])

for i in range(len(test_y)):
    if(test_y[i] == 8):
        filter_test_x.append(test_x[i])
        filter_test_y.append(test_y[i])

print(f"Length of filtered training data: {len(filter_train_y)}")
print(f"Length of filtered testing data: {len(filter_test_y)}")

Length of filtered training data: 5000
Length of filtered testing data: 1000
